In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import PIL # must do this before importing lxml because of bug in Pillow/lxml (gzip conflict)
import numpy as np
import tempfile
from itertools import islice
import xml.etree.ElementTree as ET
from lxml import etree
import io
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import imageio.v2 as imageio
import subprocess
import os
import webdataset as wds
import glob
import textcheck
import scipy.ndimage as ndi

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interact_manual
import ipywidgets as widgets

In [3]:

from importlib import reload
import iautils
reload(iautils)

<module 'iautils' from '/home/tmb/pdfocrhelpers/iautils.py'>

In [4]:
def shuffled(l):
    l = list(l)
    np.random.shuffle(l)
    return l

def checkvalid(image, text):
    assert image.ndim in [2, 3]
    if image.ndim == 3:
        assert image.shape[2] == 3
    if len(text) < 1:
        return False
    h, w = image.shape[:2]
    if h < 8 or w < 8:
        return False
    return True

In [5]:
def scale_page_(page, scale):
    page["size"]["w"] = int(page["size"]["w"] * scale)
    page["size"]["h"] = int(page["size"]["h"] * scale)
    for w in page["words"]:
        w["t"] = int(w["t"] * scale)
        w["l"] = int(w["l"] * scale)
        w["b"] = int(w["b"] * scale)
        w["r"] = int(w["r"] * scale)
    for l in page["lines"]:
        l["t"] = int(l["t"] * scale)
        l["l"] = int(l["l"] * scale)
        l["b"] = int(l["b"] * scale)
        l["r"] = int(l["r"] * scale)


In [6]:
last_display = 0

def report(pagesample=None, wordsample=None, linesample=None, every=15, title=""):
    global last_display
    import matplotlib.pyplot as plt
    from IPython import display
    if time.time()-last_display < every: return
    last_display = time.time()
    plt.close("all")
    fig = plt.figure(figsize=(12, 12))
    fig.clf()
    for i in range(4): fig.add_subplot(2, 2, i+1)    
    ax1, ax2, ax3, ax4 = fig.get_axes()
    if pagesample:
        ax1.imshow(pagesample["jpg"], cmap="gray")
        ax1.set_title(title)
        if len(pagesample["mp"]["words"]) > 0:
            for w in pagesample["mp"]["words"]:
                iautils.Bbox(from_dict=w).draw_rect("red", ax=ax1)
        else:
            for w in pagesample["mp"]["lines"]:
                iautils.Bbox(from_dict=w).draw_rect("green", ax=ax1)
        ax2.set_title(f"{len(pagesample['mp']['words'])} words, {len(pagesample['mp']['lines'])} lines")
        ax2.imshow(pagesample["lines.jpg"])
    if wordsample:
        ax3.imshow(wordsample["jpg"], cmap="gray")
        ax3.set_title(wordsample["txt"])
    if linesample:
        ax4.imshow(linesample["jpg"], cmap="gray")
        ax4.set_title(linesample["txt"])
    display.clear_output(wait=True)
    display.display(fig)

In [7]:
def extract_pages(pages, images, segwriter, wordwriter, linewriter, key, pagesel=None, wordsel=None, linesel=None, do_report=False, title=""):
    # write a usage comment in Google style for this function
    """Extract pages from an ABBYY XML-derived page structure and a list of images.

    Args:
        pages: ABBYY XML-derived page structure
        images: Jp2ZipFile object
        segwriter: TarWriter for segmentations
        wordwriter: TarWriter for word images
        linewriter: TarWriter for line images
        key: the key for the current book (used as prefix for outputs)
        pagelist: list of pages to extract. Defaults to None (=all)
        wordsel: a selection predicate for words. Defaults to None.
        linesel: a selection predicate for lines. Defaults to None.

    Returns:
        _description_
    """
    if len(images) != len(pages):
        return False

    page_offset = 0 if 0 in images else 1
    print(f"page_offset={page_offset}")

    npages, nwords, nlines = 0, 0, 0

    print(f"pages={len(pages)}:", end=" ", flush=True)

    allpages = set(range(len(pages)))
    if pagesel is not None:
        allpages = sorted(list(set(pagesel(allpages))))

    for pageno in allpages:
        print(f"{pageno} ", end="", flush=True)
        page = pages[pageno]
        page["__class__"] = "page"
        page["pageno"] = pageno
        if len(page["lines"]) == 0:
            continue
        image = images.decode(pageno+page_offset)
        h = page["size"]["h"]
        w = page["size"]["w"]
        if not (h == image.shape[0] and w == image.shape[1]):
            print("mismatch", pageno, (h, w), image.shape)
            continue
        line_mask = iautils.make_page_mask(page, page["lines"])
        word_mask = iautils.make_page_mask(page, page["words"])

        scale = 3500.0 / max(image.shape[0], image.shape[1])
        if scale < 1.0:
            image = ndi.zoom(image, (scale, scale, 1)[:image.ndim], order=1)
            line_mask = ndi.zoom(line_mask, scale, order=0)
            word_mask = ndi.zoom(word_mask, scale, order=0)
            scale_page_(page, scale)
            page["scale"] = scale

        seg = {
            "__key__": f"{key}({pageno:04d})",
            "jpg": image,
            "lines.jpg": line_mask,
            "words.jpg": word_mask,
            "mp": page,
        }
        segwriter.write(seg)
        npages += 1
        the_linesample = None
        for lineno, line in enumerate(page["lines"]):
            bbox = iautils.Bbox(from_dict=line)
            growth = 10 # max(10, bbox.height() // 10)
            lineimage = bbox.grow(growth).extract(image)
            linetext = line["text"]
            if not checkvalid(lineimage, linetext):
                continue
            if linesel is not None and not linesel(lineno, linetext, lineimage):
                continue
            linesample = {
                "__key__": f"{key}({pageno:04d}-{lineno:04d})",
                "jpg": lineimage,
                "txt": linetext,
            }
            if lineno == 5:
                the_linesample = linesample
            linewriter.write(linesample)
            nlines += 1
        the_wordsample = None
        for wordno, word in enumerate(page["words"]):
            bbox = iautils.Bbox(from_dict=word)
            growth = 10 # max(10, bbox.height() // 10)
            wordimage = bbox.grow(growth).extract(image)
            wordtext = word["text"]
            if not checkvalid(wordimage, wordtext):
                continue
            if wordsel is not None and not wordsel(wordno, wordtext, wordimage):
                continue
            wordsample = {
                "__key__": f"{key}({pageno:04d}-{wordno:04d})",
                "jpg": wordimage,
                "txt": wordtext,
            }
            if wordno == 20:
                the_wordsample = wordsample
            wordwriter.write(wordsample)
            nwords += 1
        if do_report:
            report(pagesample=seg, wordsample=the_wordsample, linesample=the_linesample, title=title)
    print()
    return npages, nwords, nlines

In [8]:
class Nothing(Exception):
    pass

In [9]:
def convert_book_shard(url, prefix="", maxbooks=99999999, pagesel=None, linesel=None, wordsel=None, exn=Nothing, do_report=False):
    ds = wds.WebDataset(url).decode()
    segwriter = wds.TarWriter(prefix+"segs.tar")
    wordwriter = wds.TarWriter(prefix+"words.tar")
    linewriter = wds.TarWriter(prefix+"lines.tar")
    nbooks = 0
    try:
        for sample in ds:
            if nbooks >= maxbooks:
                break
            key = sample["__key__"]
            print(f"key={key}")
            identifier = bytes.fromhex(key).decode("utf-8")
            print(f"identifier={identifier}")
            try:
                pages = iautils.parse_abbyy(io.StringIO(sample["abbyy.gz"].decode("utf-8")))
                if pages is None:
                    print("no pages")
                    continue
            except exn as e:
                print("failed to parse abbyy", repr(e))
                continue
            try:
                images = iautils.Jp2Zip(io.BytesIO(sample["zip"]))
            except exn as e:
                print("failed to load zip", repr(e))
                continue
            if set(images.pages.keys()) == set(range(len(pages))):
                print("zero-based page numbers in images")
            elif set(images.pages.keys()) == set(range(1, len(pages)+1)):
                print("one-based page numbers in images")
            else:
                print("page number mismatch", len(pages), len(images))
                continue
            for page in pages:
                page["identifier"] = identifier
            try:
                result = extract_pages(pages, images, segwriter, wordwriter, linewriter, key=key, pagesel=pagesel, linesel=linesel, wordsel=wordsel, do_report=do_report, title=url)
            except exn as e:
                print("failed to extract pages", repr(e))
                continue
            if result is False:
                print("failed", sample["__key__"])
            else:
                npages, nwords, nlines = result
                print(f"{prefix} {sample['__key__']} {npages} pages, {nwords} words, {nlines} lines")
                nbooks += 1
    finally:
        segwriter.close()
        wordwriter.close()
        linewriter.close()



In [10]:
def select_every(n):
    def f(i, text, image):
        return i % n == 0
    return f

def select_random_subset(n):
    def f(pages):
        pages = list(pages)
        random.shuffle(pages)
        return sorted(pages[:n])
    return f

import textcheck

In [11]:
@interact_manual
def run_test():
    convert_book_shard("books-000000.tar", prefix="test-", maxbooks=1, do_report=True, pagesel=select_random_subset(10), wordsel=textcheck.wordsel)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

In [12]:
@interact_manual
def remove_old_output():
    os.system("gsutil -m rm -r gs://ocro-iaa/segs/")
    os.system("gsutil -m rm -r gs://ocro-iaa/words/")
    os.system("gsutil -m rm -r gs://ocro-iaa/lines/")


interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

In [13]:
urls = sorted(os.popen("gsutil ls gs://ocro-iaa/books/books-*.tar").read().strip().split("\n"))
shards = [s[-10:-4] for s in urls]

In [14]:
def run(cmd):
    print("#", cmd)
    assert 0 == os.system(cmd), f"command failed: {cmd}"

In [15]:
def extract_shard(shard, do_report=False, env={}, npages=5):
    os.environ.update(env)
    print(f"\n=== {shard} ===\n")
    if os.system(f"gsutil ls gs://ocro-iaa/segs/books-{shard}-segs.tar > /dev/null 2>&1") == 0:
        print(f"shard {shard} already exists")
        return "already-done"
    url = f"gs://ocro-iaa/books/books-{shard}.tar"
    if not os.path.exists(f"books-{shard}.tar"):
        run(f"gsutil cp {url} books-{shard}.tar")
    convert_book_shard(f"./books-{shard}.tar", prefix=f"books-{shard}-", pagesel=select_random_subset(npages), wordsel=textcheck.wordsel, exn=Nothing, do_report=do_report)
    run(f"gsutil cp books-{shard}-words.tar gs://ocro-iaa/words/")
    run(f"gsutil cp books-{shard}-lines.tar gs://ocro-iaa/lines/")
    run(f"gsutil cp books-{shard}-segs.tar gs://ocro-iaa/segs/")
    os.remove(f"books-{shard}.tar")
    os.remove(f"books-{shard}-segs.tar")
    os.remove(f"books-{shard}-words.tar")
    os.remove(f"books-{shard}-lines.tar")
    return "finished"

In [16]:
credentials = {k: v for k, v in os.environ.items() if k.startswith("GOOGLE_")}

In [17]:
@interact_manual
def extract_all(do_report=True):
    for shard in shards:
        extract_shard(shard, do_report=do_report)

interactive(children=(Checkbox(value=True, description='do_report'), Button(description='Run Interact', style=…

In [18]:
@interact_manual
def extract_all_parallel():
    import ray
    import os
    if not ray.is_initialized():
        print("initializing ray")
        ray.init()
    extract_shard_ = ray.remote(num_cpus=2)(extract_shard)
    print("starting jobs")
    result = [extract_shard_.remote(shard, env=credentials) for shard in shards]
    print("waiting for jobs")
    ray.get(result)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

In [19]:
def normalize(a):
    a = a.astype("float32")
    a = a - a.min()
    a /= a.max()
    return a

In [20]:
the_bucket = "gs://"
the_shard = "000000"

In [21]:
@interact_manual
def show_segs(prefix="gs://ocro-iaa/segs/books-", shard="000000"):
    ds = wds.WebDataset(f"{prefix}{shard}-segs.tar").decode("rgb")
    sample = next(iter(islice(ds, 10, 11)))
    print(sample.keys())
    figsize(18, 10)
    subplot(131); imshow(sample["jpeg"])
    subplot(132); imshow(normalize(sample["lines.jpg"]))
    subplot(133); imshow(normalize(sample["words.jpg"]))

interactive(children=(Text(value='gs://ocro-iaa/segs/books-', continuous_update=False, description='prefix'), …

In [22]:
@interact_manual
def show_zoomed(prefix="gs://ocro-iaa/segs/books-", shard="000000"):
    ds = wds.WebDataset(f"{prefix}{shard}-segs.tar").decode("rgb")
    sample = next(iter(islice(ds, 10, 11)))
    print(sample.keys())
    figsize(18, 10)
    sl = (slice(500, 1000), slice(500, 1000))
    figsize(18, 10)
    subplot(131); imshow(sample["jpeg"][sl])
    subplot(132); imshow(normalize(sample["lines.jpg"][sl]))
    subplot(133); imshow(normalize(sample["words.jpg"][sl]))

interactive(children=(Text(value='gs://ocro-iaa/segs/books-', continuous_update=False, description='prefix'), …

In [23]:
@interact_manual
def show_lines(prefix="gs://ocro-iaa/segs/books-", shard="000000"):
    ds = wds.WebDataset(f"{prefix}{shard}-lines.tar").decode("rgb")
    src = list(islice(iter(ds), 50, 1000*17, 17))
    figsize(18, 10)
    random.shuffle(src)
    i = 0
    n = 8
    while i < n:
        sample = src[i]
        subplot(n, 1, i+1)
        imshow(sample["png"]); xticks([]); yticks([])
        title(sample["txt"])
        i += 1

interactive(children=(Text(value='gs://ocro-iaa/segs/books-', continuous_update=False, description='prefix'), …

In [24]:
@interact_manual
def show_words(prefix="gs://ocro-iaa/segs/books-", shard="000000"):
    ds = wds.WebDataset(f"{prefix}{shard}-words.tar").decode("rgb")
    src = list(islice(iter(ds), 50, 1000*17, 17))
    figsize(18, 10)
    random.shuffle(src)
    i = 0
    n = 8
    while i < n:
        sample = src[i]
        subplot(n, 1, i+1)
        imshow(sample["png"]); xticks([]); yticks([])
        title(sample["txt"])
        i += 1

interactive(children=(Text(value='gs://ocro-iaa/segs/books-', continuous_update=False, description='prefix'), …